## 2. Configuración inicial e importación de librerías

In [6]:
# Importación de librerías principales
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Opciones de visualización
plt.rcParams['figure.figsize'] = (10, 6)
pd.set_option('display.max_columns', 50)

# Estilo de gráficos
sns.set(style="whitegrid")

print("Librerías importadas correctamente.")

Librerías importadas correctamente.


## 3. Recopilación y descripción de datos

En esta sección se seleccionan los activos a analizar y se descargan sus datos históricos
(precios diarios, volumen, etc.). Posteriormente se describe el conjunto de datos resultante.

### 3.1. Selección de activos y rango temporal

Modificar las listas según sea necesario.


In [13]:
# Lista de símbolos a analizar (se pueden modificar)
assets = [
    "AAPL",    # Apple
    "MSFT",    # Microsoft
    "NVDA",    # NVIDIA
    "TSLA",    # Tesla
    "GLD"      # Oro (ETF de oro SPDR Gold Shares)
]

# Rango temporal (puede ajustarse)
start_date = "2018-01-01"
end_date = "2025-12-01"

assets, start_date, end_date

(['AAPL', 'MSFT', 'NVDA', 'TSLA', 'GLD'], '2018-01-01', '2025-12-01')

### 3.2 Descarga de Datos

In [ ]:
import os
import yfinance as yf

assets = ["AAPL", "MSFT", "NVDA", "TSLA", "GLD"]
os.makedirs("data", exist_ok=True)

for ticker in assets:
    print(f"Descargando {ticker} ...")
    df = yf.download(ticker, start=start_date, end=end_date)
    df.to_csv(f"data/{ticker}.csv")

print("✔ Descarga completa")


Borrando: data\AAPL.csv
Borrando: data\GLD.csv
Borrando: data\MSFT.csv
Borrando: data\NVDA.csv
Borrando: data\TSLA.csv
✔ Archivos antiguos eliminados
⬇️ Descargando datos de mercado...

   • Descargando AAPL...


C:\Users\carme\AppData\Local\Temp\ipykernel_14580\2643879914.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed
C:\Users\carme\AppData\Local\Temp\ipykernel_14580\2643879914.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date)


     ✔ Guardado en data/AAPL.csv
   • Descargando MSFT...


[*********************100%***********************]  1 of 1 completed
C:\Users\carme\AppData\Local\Temp\ipykernel_14580\2643879914.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date)


     ✔ Guardado en data/MSFT.csv
   • Descargando NVDA...


...

     ✔ Guardado en data/NVDA.csv
   • Descargando TSLA...


[*********************100%***********************]  1 of 1 completed
C:\Users\carme\AppData\Local\Temp\ipykernel_14580\2643879914.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date)


     ✔ Guardado en data/TSLA.csv
   • Descargando GLD...


[*********************100%***********************]  1 of 1 completed

     ✔ Guardado en data/GLD.csv

🎉 Descarga completada. Los archivos están listos en la carpeta 'data/'.


### 3.3 Carga de datos locales

En este proyecto trabajaremos **de forma local**, leyendo archivos `.csv` desde la carpeta `data/`.
Cada activo debe tener un archivo con su nombre de ticker, por ejemplo:

- `data/AAPL.csv`
- `data/MSFT.csv`
- `data/NVDA.csv`
- `data/TSLA.csv`
- `data/GLD.csv` (oro)

Los archivos deben contener al menos las columnas estándar de Yahoo Finance: `Date`, `Open`, `High`,
`Low`, `Close`, `Adj Close`, `Volume`.

> Opcionalmente se puede incluir una sección comentada con `yfinance` para descargar los datos
si se tiene conexión a internet.


In [ ]:
import os
import pandas as pd

data = {}

for ticker in assets:
    file_name = f"data/{ticker}.csv"
    if not os.path.exists(file_name):
        print(f"⚠️ Archivo no encontrado: {file_name}")
        continue

    print(f"\nCargando datos de {ticker} desde {file_name}...")

    # Leemos TODO el CSV sin asumir encabezados
    df_raw = pd.read_csv(file_name, header=None)

    # Fila 0: ['Price','Close','High','Low','Open','Volume']
    # Fila 1: ['Ticker','GLD','GLD','GLD','GLD','GLD']
    # Fila 2: ['Date','','','','','']
    # Fila 3+: fecha y valores

    # Construimos los nombres de columnas que queremos:
    # -> 'Date', 'Close', 'High', 'Low', 'Open', 'Volume'
    price_row = df_raw.iloc[0]              # fila con Price,Close,High,...
    colnames = ['Date'] + price_row[1:].tolist()

    # Nos quedamos solo con las filas de datos (a partir de la fila 3)
    df = df_raw.iloc[3:].reset_index(drop=True)
    df.columns = colnames

    # Convertimos tipos
    df["Date"] = pd.to_datetime(df["Date"])
    for c in df.columns[1:]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # Usamos la fecha como índice
    df.set_index("Date", inplace=True)

    data[ticker] = df

# Resumen
print("\nActivos cargados:", list(data.keys()))
for t, df in data.items():
    print(f"{t}: columnas -> {df.columns.tolist()}")

# Usaremos la columna 'Close' como serie principal de precios
adj_close = pd.DataFrame({ticker: df["Close"] for ticker, df in data.items()})
adj_close.head()



Cargando datos de AAPL...
  Columnas crudas: ['Price', 'Close', 'High', 'Low', 'Open', 'Volume']

Cargando datos de MSFT...
  Columnas crudas: ['Price', 'Close', 'High', 'Low', 'Open', 'Volume']

Cargando datos de NVDA...
  Columnas crudas: ['Price', 'Close', 'High', 'Low', 'Open', 'Volume']

Cargando datos de TSLA...
  Columnas crudas: ['Price', 'Close', 'High', 'Low', 'Open', 'Volume']

Cargando datos de GLD...
  Columnas crudas: ['Price', 'Close', 'High', 'Low', 'Open', 'Volume']

Activos cargados: ['AAPL', 'MSFT', 'NVDA', 'TSLA', 'GLD']
AAPL: columnas -> ['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5']
MSFT: columnas -> ['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5']
NVDA: columnas -> ['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5']
TSLA: columnas -> ['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5']
GLD: columnas -> ['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5']


KeyError: 'Close'

### 3.3. Descripción inicial del dataset

En esta sección se exploran las dimensiones del dataset, el rango de fechas, la cantidad de valores faltantes, etc.


In [ ]:
# Información básica de los datos
print("Dimensiones de adj_close:", adj_close.shape)
print("\nPrimeras filas:")
display(adj_close.head())

print("\nResumen estadístico inicial:")
display(adj_close.describe())

print("\nConteo de valores faltantes por activo:")
display(adj_close.isna().sum())

## 4. Análisis Exploratorio de Datos (EDA)

En esta sección se calculan y visualizan medidas descriptivas para comprender el comportamiento
de los activos: rendimientos, volatilidad, distribuciones, correlaciones, etc.


### 4.1. Cálculo de rendimientos diarios

In [ ]:
# Cálculo de rendimientos diarios (porcentaje)
returns = adj_close.pct_change().dropna()
returns.head()

### 4.2. Estadísticos descriptivos de los rendimientos

In [ ]:
# Estadísticos descriptivos de los rendimientos
desc_returns = returns.describe()
desc_returns

### 4.3. Histogramas de rendimientos

In [ ]:
# Histogramas de rendimientos para cada activo
returns.hist(bins=50, figsize=(12, 8))
plt.suptitle("Histogramas de rendimientos diarios", y=1.02)
plt.show()

### 4.4. Boxplots de rendimientos

In [ ]:
# Boxplots de rendimientos
plt.figure(figsize=(10, 6))
sns.boxplot(data=returns)
plt.title("Distribución de rendimientos diarios por activo")
plt.ylabel("Rendimiento diario")
plt.show()

### 4.5. Matriz de correlación y mapa de calor

In [ ]:
# Matriz de correlación de los rendimientos
corr_matrix = returns.corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", vmin=-1, vmax=1)
plt.title("Matriz de correlación de rendimientos")
plt.show()

*(Aquí se debe incluir una discusión narrativa de los hallazgos observados en el EDA,
relacionándolos con las preguntas de investigación).*

## 5. Preparación de datos

En esta sección se realizan las transformaciones necesarias para aplicar las técnicas estadísticas:
manejo de valores faltantes, creación de variables derivadas, estandarización, etc.


In [ ]:
# Eliminación de posibles filas con valores faltantes en los rendimientos
clean_returns = returns.dropna()

# Ejemplo de creación de una variable objetivo (target) para un activo concreto, por ejemplo SPY:
target_asset = "SPY"  # se puede cambiar

# Variable binaria: 1 si el rendimiento de SPY es positivo al día siguiente, 0 en caso contrario
spy_future = clean_returns[target_asset].shift(-1)
y_binary = (spy_future > 0).astype(int).dropna()

# Alineamos los datos de características con el target
X_features = clean_returns.loc[y_binary.index]

X_features.head(), y_binary.head()

### 5.1. Estandarización de variables para PCA y clustering

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(clean_returns)

X_scaled[:5]

## 6. Aplicación de técnicas estadísticas

En esta sección se aplican las técnicas estadísticas requeridas por el proyecto:  
- Pruebas de hipótesis  
- Modelos de regresión  
- Análisis de Componentes Principales (PCA)  
- Clustering  

Cada técnica debe ser justificada y sus resultados interpretados.


### 6.1. Pruebas de hipótesis

In [ ]:
# Ejemplo: t-test para comparar la media de rendimientos entre dos activos

asset1 = "AAPL"
asset2 = "MSFT"

sample1 = clean_returns[asset1]
sample2 = clean_returns[asset2]

t_stat, p_value = stats.ttest_ind(sample1, sample2, equal_var=False)  # Welch's t-test

print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")

# Aquí se debe interpretar el resultado en función de un alfa (ej. 0.05).

### 6.2. Regresión lineal y logística

In [ ]:
# --- Regresión lineal ---
# Ejemplo: modelar el rendimiento de AAPL en función del rendimiento de SPY

X_lin = clean_returns[[target_asset]]  # SPY como predictor
y_lin = clean_returns["AAPL"]          # AAPL como variable respuesta

lin_reg = LinearRegression()
lin_reg.fit(X_lin, y_lin)

print("Coeficiente:", lin_reg.coef_[0])
print("Intercepto:", lin_reg.intercept_)
print("R^2:", lin_reg.score(X_lin, y_lin))

In [ ]:
# --- Regresión logística ---
# Usamos X_features (rendimientos de todos los activos) para predecir subida/bajada de SPY

X_train, X_test, y_train, y_test = train_test_split(
    X_features, y_binary, test_size=0.2, random_state=42, shuffle=False
)

log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

y_pred = log_reg.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nMatriz de confusión:")
print(confusion_matrix(y_test, y_pred))
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

### 6.3. Análisis de Componentes Principales (PCA)

In [ ]:
pca = PCA(n_components=2)
principal_components = pca.fit_transform(X_scaled)

pc_df = pd.DataFrame(
    principal_components,
    columns=["PC1", "PC2"],
    index=clean_returns.index
)

print("Varianza explicada por cada componente:", pca.explained_variance_ratio_)

plt.figure(figsize=(8, 6))
plt.scatter(pc_df["PC1"], pc_df["PC2"], alpha=0.5)
plt.title("Proyección de los datos en los dos primeros componentes principales")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

### 6.4. Clustering (K-Means)

In [ ]:
# Clustering usando los rendimientos estandarizados
k = 3  # número de clusters (se puede ajustar)

kmeans = KMeans(n_clusters=k, random_state=42)
clusters = kmeans.fit_predict(X_scaled)

pc_df['cluster'] = clusters

plt.figure(figsize=(8, 6))
sns.scatterplot(
    data=pc_df,
    x="PC1", y="PC2",
    hue="cluster", palette="tab10", alpha=0.7
)
plt.title("Clusters en el espacio de los dos primeros componentes principales")
plt.show()

## 7. Resultados y conclusiones

En esta sección se deben sintetizar los hallazgos más importantes:

- Activo(s) con mayor volatilidad.  
- Relaciones significativas entre activos (correlaciones, regresiones).  
- Principales componentes encontrados por PCA y su interpretación.  
- Estructura de clusters y qué significan en términos de comportamiento de los activos.  
- Desempeño del modelo de regresión logística para predecir subidas/bajadas.  

Finalmente, se responden explícitamente las preguntas de investigación planteadas en la sección 1.3.


## 8. Limitaciones y trabajo futuro

- (Describir las limitaciones del análisis: rango temporal, tipo de modelo, ausencia de información macroeconómica, etc.).  
- (Proponer extensiones: incluir más activos, usar otros modelos de series temporales, etc.).


## 9. Referencias

- Apuntes y materiales del curso de Estadística.  
- Documentación de `pandas`, `matplotlib`, `seaborn`, `scikit-learn`.  
- Yahoo Finance / yfinance para datos financieros.  
